In [3]:
import requests
import dateutil.parser
from datetime import timedelta, datetime
import pandas as pd  # Import the pandas library

# Function to get the data from the API
def get_supply_data(start_date, end_date, api_key):
    url = f"https://web3api.io/api/v2/market/metrics/eth/supply/historical?timeFormat=iso&startDate={start_date}&endDate={end_date}"
    headers = {
        "accept": "application/json",
        "x-api-key": api_key
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json().get('payload', {}).get('data', [])
    else:
        print(f"Failed to retrieve data: {response.status_code} - {response.text}")
        return None

# Function to calculate inflation rates
def calculate_inflation_rates(data):
    previous_month_supply = None
    previous_month_timestamp = None
    inflation_rates = []

    for entry in data:
        timestamp = dateutil.parser.isoparse(entry['timestamp'])
        circulating_supply = float(entry['circulatingSupply'])

        if previous_month_supply is None:
            previous_month_supply = circulating_supply
            previous_month_timestamp = timestamp
        else:
            if timestamp.month != previous_month_timestamp.month or timestamp.year != previous_month_timestamp.year:
                inflation_rate = ((circulating_supply - previous_month_supply) / previous_month_supply) * 100
                inflation_rates.append((previous_month_timestamp.strftime("%Y-%m"), inflation_rate))
                previous_month_supply = circulating_supply
                previous_month_timestamp = timestamp
    return inflation_rates

# Your actual API key
api_key = "API-KEY-HERE"

# Date range for the past 3 years
end_date = datetime.now()
start_date = end_date - timedelta(days=3*365)

# Accumulated data
accumulated_data = []

# Loop over each period
while start_date < end_date:
    next_end_date = min(start_date + timedelta(days=183), end_date)
    data = get_supply_data(start_date.date().isoformat(), next_end_date.date().isoformat(), api_key)
    if data:
        accumulated_data.extend(data)
    start_date = next_end_date

# Calculate the inflation rates
inflation_rates = calculate_inflation_rates(accumulated_data)

# Create a DataFrame from the inflation rates
df = pd.DataFrame(inflation_rates, columns=['Month', 'Inflation Rate'])
styled_df = df.style.format({"Inflation Rate": "{:.4f}%"}).set_caption("Monthly Ethereum Inflation Rate %")
display(styled_df)

,Month,Inflation Rate
0,2020-11,0.3479%
1,2020-12,0.3554%
2,2021-01,0.3677%
3,2021-02,0.3308%
4,2021-03,0.3649%
5,2021-04,0.3515%
6,2021-05,0.3611%
7,2021-06,0.3467%
8,2021-07,0.3540%
9,2021-08,0.3567%
